In [ ]:

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import numpy as np

# Usar renderer para notebooks (más rápido que browser)
pio.renderers.default = "notebook"

In [ ]:
def load_mock_data():
    # Datos del Funnel
    funnel_data = {
        'Etapa': ['Usuarios', '/registro', 'Registro Finalizado'],
        'Cantidad': [35215, 3715, 615]
    }
    
    # Evolución Temporal (1 al 7 de Enero 2026)
    dates = pd.date_range(start="2026-01-01", end="2026-01-07")
    evolution_data = pd.DataFrame({
        'Fecha': dates,
        'Intención de Registro': [510, 780, 650, 720, 810, 1020, 1080], # Mock de sesiones
        'Registro': [45, 68, 95, 82, 195, 120, 360]
    })
    
    # Datos por Dispositivo
    device_data = pd.DataFrame({
        'Dispositivo': ['Mobile', 'Desktop', 'Tablet', 'Smart TV'],
        'Intención': [1450, 1380, 180, 100],
        'Registro': [380, 240, 20, 5]
    })

    return funnel_data, evolution_data, device_data

funnel, evolution, devices = load_mock_data()


In [ ]:
valores= funnel['Cantidad']
etapas = funnel['Etapa']

# 2. Cálculos manuales
# Porcentaje respecto al total (Paso 1)
pct_initial = [(v / valores[0]) * 100 for v in valores]
# Porcentaje respecto al paso anterior (opcional, muy útil)
pct_previous = [100.0] + [(valores[i] / valores[i-1]) * 100 for i in range(1, len(valores))]

# 3. Crear etiquetas de texto personalizadas
# Formato: "35,215,664 (100%)"
textos_personalizados = [
    f"{v:,}<br>{p:.2f}%" if p < 100 else f"{v:,}" 
    for v, p in zip(valores, pct_initial)
]


fig = go.Figure(go.Funnel(
    y = etapas,
    x = np.log10(valores),
    text = textos_personalizados,
    textinfo = "text", 
    textposition = "inside",
    insidetextanchor = "middle",
    marker = {
        # "color": ["#2450A6", "#1E43E6", "#1E88E6"],
        "color": ["#2450A6", "#1565C0", "#4E8ACF"],
        "line": {"width": 1, "color": "white"}
    },
    connector = {"fillcolor": "#A6C6ED", "line": {"width": 0}},
    hoverinfo = "y+text", # Al pasar el mouse muestra Etapa + Valor Real
))

fig.update_layout(
    title = {"text": "<b>Embudo de Conversión - Infobae</b><br><span style='font-size:12px'>Escala logarítmica aplicada para visibilidad</span>"},
    # paper_bgcolor = "white",
    # plot_bgcolor = "white",
    xaxis = {
        "showticklabels": False, # Escondemos los números del logaritmo (2, 4, 6, 8)
        "showgrid": False,
        "zeroline": False
    },
    margin = dict(l=200), # Espacio para que no se corten los nombres de las etapas
    height = 500
)

fig.show()



In [117]:
# === MAPA DE USUARIOS POR PAÍS ===

# Datos mock por país
paises_data = pd.DataFrame({
    'Pais': ['Argentina', 'México', 'España', 'Colombia', 'Chile', 'Perú'],
    'ISO': ['ARG', 'MEX', 'ESP', 'COL', 'CHL', 'PER'],  # Códigos ISO para el mapa
    'Usuarios': [15000, 8500, 4200, 3100, 2800, 1900],
    'Registros': [2500, 1200, 680, 450, 380, 290]
})

# Mapa Choropleth (países coloreados por métrica)
fig_mapa = px.choropleth(
    paises_data,
    locations='ISO',               # Columna con códigos de país
    color='Usuarios',              # Métrica para colorear
    hover_name='Pais',             # Nombre al pasar mouse
    hover_data=['Registros'],      # Datos adicionales en hover
    color_continuous_scale=[[0, '#FEF0E3'], [0.5, '#F9B86C'], [1, '#F28322']] ,  # Escala de colores
    projection='natural earth',    # Tipo de proyección
    title='📍 Usuarios por País'
)

fig_mapa.update_layout(
    geo=dict(
        showframe=False,
        showcoastlines=True,
        coastlinecolor='lightgray',
        showland=True,
        countrywidth=1,
        landcolor='#f5f5f5',
        showcountries=True,
        countrycolor='lightgray',
         showlakes=False,            # Sin lagos
        showrivers=False,
         lataxis=dict(showgrid=False, gridcolor='white'),  # Sin cuadrícula latitud
        lonaxis=dict(showgrid=False, gridcolor='white'),  # Sin cuadrícula longitud
        showsubunits=False,         # Sin subdivisiones internas
        subunitcolor='white'
    ),
    height=500,
    margin=dict(l=0, r=0, t=50, b=0)
)

fig_mapa.show()


In [98]:
# === MAPA CON BURBUJAS POR PAÍS ===

# Datos con coordenadas centrales de cada país
paises_puntos = pd.DataFrame({
    'Pais': ['Argentina', 'México', 'España', 'Colombia', 'Chile', 'Perú'],
    'Lat': [-38.4, 23.6, 40.4, 4.6, -35.7, -9.2],
    'Lon': [-63.6, -102.5, -3.7, -74.1, -71.5, -75.0],
    'Usuarios': [15000, 8500, 4200, 3100, 2800, 1900],
    'Registros': [2500, 1200, 680, 450, 380, 290]
})

fig_puntos = px.scatter_geo(
    paises_puntos,
    lat='Lat',
    lon='Lon',
    size='Usuarios',              # Tamaño de burbuja según usuarios
    color='Registros',            # Color según registros
    hover_name='Pais',
    hover_data=['Usuarios', 'Registros'],
    projection='natural earth',
    title='📍 Usuarios por País (Burbujas)',
    size_max=50,                  # Tamaño máximo de burbuja
    color_continuous_scale=[[0, '#FEF0E3'], [0.5, '#F9B86C'], [1, '#F28322']]  # Escala naranja personalizada
)

fig_puntos.update_layout(
    geo=dict(
        showframe=False,
        showcoastlines=False,       # Sin línea de costa
        showland=True,
        landcolor='#f5f5f5',
        oceancolor='#e6f2ff',
        showcountries=True,
        countrycolor='#cccccc',    # Gris claro (podés cambiar a otro color)
        countrywidth=1,
        showlakes=False,            # Sin lagos
        showrivers=False,           # Sin ríos
        lataxis=dict(showgrid=False, gridcolor='white'),  # Sin cuadrícula latitud
        lonaxis=dict(showgrid=False, gridcolor='white'),  # Sin cuadrícula longitud
        showsubunits=False,         # Sin subdivisiones internas
        subunitcolor='white'
    ),
    height=500,
    margin=dict(l=0, r=0, t=50, b=0)
)

fig_puntos.show()
